In [21]:
import os
from dotenv import load_dotenv
from openai import OpenAI
from googleapiclient.discovery import build


import httpx

class CustomHTTPClient(httpx.Client):
    def __init__(self, *args, **kwargs):
        kwargs.pop("proxies", None) # Remove the 'proxies' argument if present
        super().__init__(*args, **kwargs)

load_dotenv()
client = OpenAI(api_key=os.environ['API_KEY'], http_client=CustomHTTPClient())
model_name = "gpt-4o-mini-2024-07-18"

In [22]:
import json
import tiktoken # for token counting
import numpy as np
from collections import defaultdict

In [23]:
data_path = "training.jsonl"

# Load the dataset
with open(data_path, 'r', encoding='utf-8') as f:
    dataset = [json.loads(line) for line in f]

# Initial dataset stats
print("Num examples:", len(dataset))
print("First example:")
for message in dataset[0]["messages"]:
    print(message)

Num examples: 10
First example:
{'role': 'user', 'content': '폭삭 속았수다'}
{'role': 'assistant', 'content': '정말 수고많으셨습니다'}


In [24]:
# Format error checks
format_errors = defaultdict(int)

for ex in dataset:
    if not isinstance(ex, dict):
        format_errors["data_type"] += 1
        continue
        
    messages = ex.get("messages", None)
    if not messages:
        format_errors["missing_messages_list"] += 1
        continue
        
    for message in messages:
        if "role" not in message or "content" not in message:
            format_errors["message_missing_key"] += 1
        
        if any(k not in ("role", "content", "name", "function_call") for k in message):
            format_errors["message_unrecognized_key"] += 1
        
        if message.get("role", None) not in ("system", "user", "assistant", "function"):
            format_errors["unrecognized_role"] += 1
            
        content = message.get("content", None)
        function_call = message.get("function_call", None)
        
        if (not content and not function_call) or not isinstance(content, str):
            format_errors["missing_content"] += 1
    
    if not any(message.get("role", None) == "assistant" for message in messages):
        format_errors["example_missing_assistant_message"] += 1

if format_errors:
    print("Found errors:")
    for k, v in format_errors.items():
        print(f"{k}: {v}")
else:
    print("No errors found")

No errors found


In [26]:
# 学習データのアップロード
uploaded_file = client.files.create(
  file=open("training.jsonl", "rb"),
  purpose="fine-tune"
)

In [28]:
import time

# 5. 파인튜닝 job 생성
print("🚀 파인튜닝 Job 생성 중...")
fine_tune_job = client.fine_tuning.jobs.create(
    training_file=uploaded_file.id,
    model=model_name
)
job_id = fine_tune_job.id
print("✅ Job 생성됨! Job ID:", job_id)

# 6. Job 상태 모니터링
print("⏳ 파인튜닝 진행 상황 확인 중...")
while True:
    job_status = client.fine_tuning.jobs.retrieve(job_id)
    status = job_status.status
    print(f"🔄 현재 상태: {status}")
    if status in ["succeeded", "failed", "cancelled"]:
        break
    time.sleep(10)  # 10초 간격으로 확인

🚀 파인튜닝 Job 생성 중...
✅ Job 생성됨! Job ID: ftjob-8k3iLpUtpbir34cFEECi04uz
⏳ 파인튜닝 진행 상황 확인 중...
🔄 현재 상태: validating_files
🔄 현재 상태: validating_files
🔄 현재 상태: validating_files
🔄 현재 상태: validating_files
🔄 현재 상태: running
🔄 현재 상태: running
🔄 현재 상태: running
🔄 현재 상태: running
🔄 현재 상태: running
🔄 현재 상태: running
🔄 현재 상태: running
🔄 현재 상태: running
🔄 현재 상태: running
🔄 현재 상태: running
🔄 현재 상태: running
🔄 현재 상태: running
🔄 현재 상태: running
🔄 현재 상태: running
🔄 현재 상태: running
🔄 현재 상태: running
🔄 현재 상태: running
🔄 현재 상태: running
🔄 현재 상태: running
🔄 현재 상태: running
🔄 현재 상태: running
🔄 현재 상태: running
🔄 현재 상태: running
🔄 현재 상태: running
🔄 현재 상태: running
🔄 현재 상태: running
🔄 현재 상태: running
🔄 현재 상태: running
🔄 현재 상태: running
🔄 현재 상태: succeeded


In [29]:
client.fine_tuning.jobs.retrieve(job_id)

FineTuningJob(id='ftjob-8k3iLpUtpbir34cFEECi04uz', created_at=1744354356, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt-4o-mini-2024-07-18:personal::BL2ctnYj', finished_at=1744354681, hyperparameters=Hyperparameters(batch_size=1, learning_rate_multiplier=1.8, n_epochs=10), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-BEfrLC4bCyHxjR7Ri25802FM', result_files=['file-CstdjLXrMbb2B71u25Hzrq'], seed=2135810063, status='succeeded', trained_tokens=1720, training_file='file-NujbkYXpVEyNKRfXt6HRPh', validation_file=None, estimated_finish=None, integrations=[], metadata=None, method=Method(dpo=None, supervised=MethodSupervised(hyperparameters=MethodSupervisedHyperparameters(batch_size=1, learning_rate_multiplier=1.8, n_epochs=10)), type='supervised'), user_provided_suffix=None)

In [31]:
# プロンプトの設定
content = "제주 방언 '폭삭 속았수다'의 의미에 대해 알고 싶어"

# APIへリクエスト
response = client.chat.completions.create(
  model="ft:gpt-4o-mini-2024-07-18:personal::BL2ctnYj",
  messages=[
    {"role": "user", "content": content}
  ]
)

# 言語モデルからの回答を表示
print(response.choices[0].message.content.strip())

제주 방언 '폭삭 속았수다'는 "정말 수고많으셨습니다"라는 의미입니다.
